# Cryptocurrency Dashboard

## Code Structure

**I.** **Import** 

**II.** **Initialize Panel** 

**III.** **Constants & Variables** 

- Binance API Keys & URLs 
- Cryptocurrency Symbols
- Symbol Colors for Plotly
- Moving Averages Periods
- Time Intervals **NEW**

**IV.** **Helper Functions** 
- Fetch Historical Price Data from Binance
- Fetch Top Symbols for ETH- & BNB-Chain **NEW**
- Filter Outliers
- Calculate All-Time-High
- Calculate High-Lows **NEW**
- Calculate Trendline **NEW**
- Fetch, Save & Merge Price Data to CSV Files
- Load Combined Data from CSV File
- Load Symbol Data from CSV Files
- Fetch Current Price from Binance
- Convert Colors for Plotly

**V.** **Main Work Flow**
- Global Variables
- Main Function
- Run Main

**VI.** **Figures**
- Resources for Styling
- Current Price vs. All-Time High for All Symbols
- Stacked Price Curves for All Symbols
- Simple Price Over Time **NEW**
- Detailed Price Over Time
- Simple Volume Over Time
- Detailed Volume Over Time **NEW**
  

**VII.** **Dashboard**
- Resources
- Dashboard Components
- Dashboard Creation

## I. Import

In [4]:
# Import Libraries

# General
import os
import requests
from dotenv import load_dotenv

# Binance API
from binance.client import Client

# Data
import pandas as pd

# Calculations
from datetime import datetime, timedelta
import numpy as np

# Dashboard & Widgets
import panel as pn

# Plots & Colors
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.colors as mcolors

## II. Initialize Panel & Figure Rendering Settings

In [5]:
pn.extension('tabulator', 'plotly', sizing_mode="stretch_width")

In [88]:
# print(pio.renderers)

In [89]:
# pio.renderers.default = "plotly_mimetype+notebook_connected"

## III. Constants & Variables

### Binance API

In [46]:
# Load API Key from .env file
load_dotenv('keys.env')
BINANCE_API_KEY = os.getenv('BINANCE_API_KEY')

# Binance API 
BINANCE_API_URL = 'https://api.binance.com/api/v3/klines'
BINANCE_API_URL_CURRENT_PRICE = 'https://api.binance.com/api/v3/ticker/price?symbol='

In [47]:
client = Client(BINANCE_API_KEY)

### Cryptocurrency Symbols

In [48]:
# Symbols
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']

### Symbol Colors for Plotly

In [49]:
# Colors
colors_a = {'BTC': 'orange', 'ETH': 'mediumpurple', 'BNB': 'indianred'}
colors_b = {'BTC': 'gold', 'ETH': 'plum', 'BNB': 'lightsalmon'}

### Moving Averages Periods

In [50]:
# Moving Average Periods
short_window = 50  # 50-day Moving Average
long_window = 200  # 200-day Moving Average

### Time Intervals

In [51]:
# Time Intervals
intervals = {
    'All_Time': [None, None],
    '5Y': [int((datetime.now() - timedelta(days=5*365)).timestamp() * 1000), None],
    '1Y': [int((datetime.now() - timedelta(days=365)).timestamp() * 1000), None],
    '6M': [int((datetime.now() - timedelta(days=180)).timestamp() * 1000), None],
    '3M': [int((datetime.now() - timedelta(days=90)).timestamp() * 1000), None],
    '1M': [int((datetime.now() - timedelta(days=30)).timestamp() * 1000), None],
    '2W': [int((datetime.now() - timedelta(days=14)).timestamp() * 1000), None],
    '1W': [int((datetime.now() - timedelta(days=7)).timestamp() * 1000), None],
}

## IV. Helper Functions

### Fetch Historical Price Data from Binance

The `fetch_historical_data` function retrieves historical price data for a given cryptocurrency symbol from the Binance API.

In [52]:
# Fetch Historical Price Data from Binance

def fetch_historical_data(symbol='BTCUSDT', interval='1d', start_time=None, end_time=None, limit=5000):
    """Fetch historical data for a given symbol from Binance API."""
    
    # Parameters to be sent in the request to Binance API
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit,
        'startTime': start_time,
        'endTime': end_time
    }
    
    # Send API request
    response = requests.get(BINANCE_API_URL, headers={'X-MBX-APIKEY': BINANCE_API_KEY}, params=params)

    # Check Response Status
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None

    # Process Data
    data = response.json()
    df = pd.DataFrame(data, columns=[
        'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume',
        'Close Time', 'Quote Asset Volume', 'Number of Trades',
        'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
    ])

    # Edit DataFrame
    
    # Convert datatypes
    df['Date'] = pd.to_datetime(df['Open Time'], unit='ms')
    df['High'] = pd.to_numeric(df['High'])
    df['Low'] = pd.to_numeric(df['Low'])
    df['Open'] = pd.to_numeric(df['Open'])
    df['Close'] = pd.to_numeric(df['Close'])
    df['Volume'] = pd.to_numeric(df['Volume'])
    
    # Drop irrelevant columns
    df = df.drop(columns=['Open Time', 'Close Time', 'Quote Asset Volume',
                          'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'])
    # Add Symbol Column (drop USDT from pair)
    df['Symbol'] = symbol[:-4]

    # Calculate Moving Averages
    if len(df) < long_window:
        print(f"Not enough data to calculate long-term moving averages for {symbol}")

    else:
        # Calculate Simple Moving Averages (SMA)
        df['SMA_50'] = df['Close'].astype(float).rolling(window=short_window).mean()
        df['SMA_200'] = df['Close'].astype(float).rolling(window=long_window).mean()
        
        # Calculate Exponential Moving Averages (EMA)
        df['EMA_50'] = df['Close'].astype(float).ewm(span=short_window, adjust=False).mean()
        df['EMA_200'] = df['Close'].astype(float).ewm(span=long_window, adjust=False).mean()
        
        # Handle Missing Columns
        if 'SMA_50' not in df.columns:
            df['SMA_50'] = pd.Series([None] * len(df))

        if 'SMA_200' not in df.columns:
            df['SMA_200'] = pd.Series([None] * len(df))
        
        if 'EMA_50' not in df.columns:
            df['EMA_50'] = pd.Series([None] * len(df))
        
        if 'EMA_200' not in df.columns:
            df['EMA_200'] = pd.Series([None] * len(df))
    
    # Return DataFrame
    return df

#### Test and Debug

In [53]:
# Test and Debug
symbol = 'BTCUSDT'
df = fetch_historical_data(symbol=symbol)
if df is None:
    print(f"No data fetched for {symbol}")
else:
    print(df[['Close', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200']].tail(10))

         Close      SMA_50      SMA_200        EMA_50       EMA_200
990  104103.72  89633.1694  91787.70185  92931.031902  87518.982779
991  103507.82  89955.4684  91969.77715  93345.807906  87678.075686
992  103763.71  90292.5592  92148.48720  93754.353086  87838.131749
993  103463.90  90617.1970  92315.99565  94135.119632  87993.612030
994  103126.65  90991.2424  92467.94680  94487.728666  88144.189522
995  106454.26  91467.3568  92638.49440  94957.004404  88326.379278
996  105573.74  91931.0318  92814.90305  95373.346977  88497.994808
997  106849.99  92417.0314  93001.67295  95823.411409  88680.601725
998  109643.99  92906.7444  93203.01920  96365.394883  88889.192653
999  111181.83  93480.0552  93415.04840  96946.431554  89111.009940


#### Function Parameters

`symbol='BTCUSDT'`: The trading pair you want to fetch data for, with a default value of 'BTCUSDT' (Bitcoin to US Dollar Tether). 

`interval='1d'`: The time interval for each data point, with a default of '1 day' (daily candlesticks).

`start_time=None`: The starting timestamp for the data. If not specified, it defaults to the earliest available data.

`end_time=None`: The ending timestamp for the data. If not specified, it defaults to the most recent available data.

`limit=1000`: The maximum number of data points to return. The default is 1000, but Binance API limits might impose other constraints.

### Fetch Top 10 Altcoins by Chain

In [54]:
def fetch_top_tokens_by_chain(chain: str, limit: int = 10, exclude_stablecoins=True):
    # List of stablecoin suffixes to exclude
    stablecoins = ['USDT', 'USDC', 'BUSD', 'TUSD', 'DAI', 'USDP', 'USDD', 'FDUSD', 'USD', 'EUR']
    tickers = client.get_ticker()
    
    # Filter for the pairs that belong to the given chain (ETH or BNB)
    if chain.upper() == 'ETH':
        filtered_pairs = [
            ticker['symbol'] for ticker in tickers
            if 'ETH' in ticker['symbol'] and not (
                exclude_stablecoins and any(ticker['symbol'].endswith(q) for q in stablecoins)
            )
        ]
    elif chain.upper() == 'BNB':
        filtered_pairs = [
            ticker['symbol'] for ticker in tickers
            if 'BNB' in ticker['symbol'] and not (
                exclude_stablecoins and any(ticker['symbol'].endswith(q) for q in stablecoins)
            )
        ]
    else:
        raise ValueError(f"Unsupported chain: {chain}")

    # Sort by trading volume or any other metric, for example, 24h quote volume
    sorted_pairs = sorted(tickers, key=lambda x: float(x['quoteVolume']), reverse=True)
    
    # Extract the symbols of the sorted pairs, and take the top 'limit' ones
    top_pairs = [pair['symbol'] for pair in sorted_pairs if pair['symbol'] in filtered_pairs][:limit]
    
    return top_pairs


#### Test and Debug

In [55]:
# Fetch the top 10 ETH-based tokens EXCLUDING stablecoin pairs
eth_top_tokens = fetch_top_tokens_by_chain('ETH', limit=10)
print("Top 10 ETH tokens (no stables):", eth_top_tokens)

Top 10 ETH tokens (no stables): ['ETHJPY', 'ETHTRY', 'ETHFITRY', 'ETHBRL', 'ETHARS', 'ETHEURI', 'ETHPLN', 'ETHMXN', 'ETHZAR', 'WBETHETH']


In [56]:
# Fetch the top 10 BNB-based tokens EXCLUDING stablecoin pairs
bnb_top_tokens = fetch_top_tokens_by_chain('BNB', limit=10)
print("Top 10 BNB tokens (no stables):", bnb_top_tokens)

Top 10 BNB tokens (no stables): ['BNBJPY', 'BNBTRY', 'BNBARS', 'BNBBRL', 'SOLBNB', 'BNBETH', 'ALTBNB', 'SUIBNB', 'XRPBNB', 'CTKBNB']


### Filter Outliers

The `filter_outliers_percentiles` function filters a column for outliers to remove buggy data. 

In [57]:
def filter_outliers_percentiles(df, column_name, lower_percentile=0.01, upper_percentile=99.99):
    lower_bound = df[column_name].quantile(lower_percentile / 100)
    upper_bound = df[column_name].quantile(upper_percentile / 100)
    return df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

#### Function Parameters

`df`: **Set dataframe** to look at, when calling the function. 

`column_name`: **Set column name** to look at, when calling the function.

`lower_percentile=0.01`, 
`upper_percentile=99.99`: 

**Upper and Lower boundaries to filter.** Set at 0.01% and 99.99% to filter only obviously atypical data, since analyzing outliers can be important for cryptocurrency market data.


#### Show , Filter & Compare Dataframe

Check retrieved data before working with it.

In [58]:
symbol = 'BTCUSDT'
df = fetch_historical_data(symbol=symbol)
df_filtered = filter_outliers_percentiles(df, 'High')

df_widget_1 = pn.widgets.DataFrame(df, name='DataFrame')
df_widget_2 = pn.widgets.DataFrame(df_filtered, name='Filtered DataFrame')

pn.Row( df_widget_1, df_widget_2).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'8cbc2261-6e60-45b2-9791-3f2deac4c827': {'version…

### Calculate All-Time-High

The `calculate_ath` function filters the data for the **highest High Price** for a `symbol`.

**Note:** \
This value is equivalent to the **All-Time-High** if `Start Time` and `End Time` parameters of the `fetch_historical_data` function are set at `None`. 

Only Binance data is used here, so it can vary from the general market All-Time-High for a symbol.

If the value is obvioulsly wrong (like BTC over 100k when everywhere else is 70k), the `filter_outliers_percentage` function can be used to remove such values from the dataframe.



In [59]:
def calculate_ath(symbol):
    df = fetch_historical_data(symbol)
    df_filtered_percentiles_high = filter_outliers_percentiles(df, 'High')
    ath = df_filtered_percentiles_high['High'].max()
    return ath

In [60]:
# Example usage
btc_ath = calculate_ath('BTCUSDT')
print(f"Calculated BTC ATH: $ {btc_ath}")

Calculated BTC ATH: $ 110797.38


### Calculate Trendline (Linear Regression)

In [61]:
# def calculate_trendline(df):
#     # Prepare data for regression
#     df = df[['Date', 'Close']].dropna()
#     df['Date_Ordinal'] = df['Date'].apply(lambda x: x.toordinal())  # Convert date to ordinal for regression
    
#     # Linear regression using numpy's polyfit
#     X = df['Date_Ordinal'].values
#     y = df['Close'].values
    
#     # Perform linear regression (1-degree polynomial)
#     slope, intercept = np.polyfit(X, y, 1)
    
#     # Calculate trendline values
#     df['Trendline'] = slope * X + intercept
    
#     return df, (slope, intercept)

In [62]:
# # Example usage with a sample dataframe:
# data = {'Date': pd.date_range(start='2023-01-01', periods=100),
#         'Close': np.random.rand(100) * 100}

# df = pd.DataFrame(data)
# df_with_trendline, model = calculate_trendline(df)

# print(df_with_trendline.head())
# print("Model slope and intercept:", model)

### Fetch, Save & Merge Price Data to CSV Files

The `fetch_and_save_all_time_data` function is designed to **fetch historical data for multiple cryptocurrency symbols** and **save the data into CSV files**. 

**Function Purpose** 
- Retrieve historical data for a list of cryptocurrency symbols. 
- Save the data to a CSV file for each cryptocurrency symbol (**BTCUSDT.csv**, etc.). 
- Combine all retrieved data into a single Pandas DataFrame. 
- Save the combined DataFrame to a CSV file (**combined_data.csv**).

In [63]:
def fetch_and_save_all_time_data(symbols, output_dir='data', final_csv_path='combined_data.csv'):
    """Fetch and save all-time data for a list of symbols, then merge the files into one CSV."""

    # Create Output Directory for CSVs
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Fetch and save data for each symbol
    for symbol in symbols:
        # Call function to fetch data from Binance API
        df = fetch_historical_data(symbol=symbol)
        if df is not None:
            # Generate a file path for each symbol's data
            file_path = os.path.join(output_dir, f'{symbol}.csv')
            # Save to CSV-File
            df.to_csv(file_path, index=False)
            # Print Success Message (Debug)
            print(f"Data for {symbol} saved to {file_path}")
        else:
            # Print Error Message (Debug)
            print(f"No data available for {symbol}")

    # List all CSV files in the output directory
    csv_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.csv')]

    # Combine all CSV files into a single DataFrame
    if csv_files:
        # Combine dataframes
        combined_df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)
        # Save combined dataframe
        combined_df.to_csv(final_csv_path, index=False)
        # Print Success Message (Debug)
        print(f"All data combined and saved to {final_csv_path}")
    else:
        # Print Error Message (Debug)
        print("No CSV files found to combine.")



##### Test and Debug

In [64]:
# Example usage:
fetch_and_save_all_time_data(symbols)

Data for BTCUSDT saved to data/BTCUSDT.csv
Data for ETHUSDT saved to data/ETHUSDT.csv
Data for BNBUSDT saved to data/BNBUSDT.csv
All data combined and saved to combined_data.csv


##### Function Parameters

- `output_dir='data'`: Output directory for all-time price data for each Symbol.
- `final_csv_path='combined_data.csv'`: Merged data for all symbols fetched.

### Load Combined data from CSV

The `load_combined_data_from_csv` is designed to **load the merged data from the combined CSV-File** and **return a DataFrame** for futher analysis.

In [65]:
# Load All-Time Data from CSV

def load_combined_data_from_csv(final_csv_path='combined_data.csv'):
    """Load all-time data from a CSV file."""
    if os.path.exists(final_csv_path):
        df = pd.read_csv(final_csv_path)
        # Verify SMA and EMA columns
        required_columns = ['SMA_50', 'SMA_200', 'EMA_50', 'EMA_200']
        for col in required_columns:
            if col not in df.columns:
                print(f"Column {col} not found in {symbol} data. Recalculating...")
                df[col] = None  # Placeholder if recalculating is needed
        return df
    else:
        print(f"{final_csv_path} does not exist.")
        return None

#### Example Usage & Debug

In [66]:
# Example usage:
fetch_and_save_all_time_data(symbols)
combined_df = load_combined_data_from_csv()

Data for BTCUSDT saved to data/BTCUSDT.csv
Data for ETHUSDT saved to data/ETHUSDT.csv
Data for BNBUSDT saved to data/BNBUSDT.csv
All data combined and saved to combined_data.csv


In [70]:
# Debug
if combined_df is not None:
    print(f"Data preview: {combined_df.head()}") # Check the data being passed
else:
    print("No data loaded from CSV.")

Data preview:       Open     High      Low    Close        Volume  Number of Trades  \
0  1508.18  1518.42  1447.52  1491.06  7.561367e+05         1015752.0   
1  1491.06  1511.00  1424.00  1426.76  5.448464e+05          756843.0   
2  1426.76  1559.99  1422.08  1551.80  8.262521e+05         1099487.0   
3  1551.79  1606.13  1473.61  1524.59  1.003075e+06         1292499.0   
4  1524.59  1619.78  1524.59  1554.10  9.492277e+05         1471691.0   

         Date Symbol  SMA_50  SMA_200       EMA_50      EMA_200  
0  2022-08-27    ETH     NaN      NaN  1491.060000  1491.060000  
1  2022-08-28    ETH     NaN      NaN  1488.538431  1490.420199  
2  2022-08-29    ETH     NaN      NaN  1491.019277  1491.030943  
3  2022-08-30    ETH     NaN      NaN  1492.335776  1491.364864  
4  2022-08-31    ETH     NaN      NaN  1494.757903  1491.989094  


### Read CSV Data (for Debugging)

The `get_csv_data` is a more general function, that can be modified to **read data from a specific CSV-File**.

In [71]:
@pn.cache
def get_csv_data():
  return pd.read_csv('combined_data.csv', parse_dates=["Date"], index_col="Date")

datacheck = get_csv_data()

datacheck.tail()

,Open,High,Low,Close,Volume,Number of Trades,Symbol,SMA_50,SMA_200,EMA_50,EMA_200
Date,,,,,,,,,,,
NaT,NaN,NaN,NaN,NaN,NaN,NaN,BNBETH,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,ALTBNB,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,SUIBNB,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,XRPBNB,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,CTKBNB,NaN,NaN,NaN,NaN


### Load Symbol Data from CSVs

The `load_symbol_data` function is designed to **read the symbol CSV-Files** and **return DataFrames for each symbol**.

In [72]:
def load_symbol_data(symbols, directory='data'):
    dataframes = {}

    for symbol in symbols:
        # Construct the file path for the CSV file
        file_path = os.path.join(directory, f'{symbol}.csv')

        try:
            # Try to load the CSV if the file exists
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)

                # Remove 'USDT' from symbol to create a key for the dataframe
                df_name = symbol.replace('USDT', '')

                # Store the DataFrame in the dictionary
                dataframes[df_name] = df
            else:
                # Inform the user if the file doesn't exist
                print(f"File {file_path} does not exist.")
        
        except Exception as e:
            print(f"An error occurred while loading {file_path}: {e}")
    
    return dataframes

#### Example Usage & Debug

In [73]:
# Example usage:
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']  # List of symbols
dataframes = load_symbol_data(symbols)

In [74]:
# Accessing a specific DataFrame
df_BTC = dataframes.get('BTC')  # Example of accessing the DataFrame for BTC

In [75]:
# Debug & Display
df_widget_BTC = pn.widgets.DataFrame(df_BTC, name='DataFrame')
df_widget_BTC

BokehModel(combine_events=True, render_bundle={'docs_json': {'acea22ae-a43c-4ffa-9b57-a08c5350a965': {'version…

### Fetch Current Price from Binance API

Generic function to fetch the current price for a currency from Binance.

In [76]:
# Fetch Current Price from Binance

def fetch_current_price(symbol):
    """Fetch the current price for a given symbol from Binance API."""
    url = BINANCE_API_URL_CURRENT_PRICE + symbol
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return float(data.get('price', 0))
    else:
        print(f"Error fetching current price for {symbol}: {response.status_code}")
        return None

### Convert Colors for Plotly

Function to convert named colors (matplotlib) to RGBA for Plotly.

In [77]:
# Convert Colors for Plotly

def convert_color(color_name, opacity=0.8):
    """Convert a color name to rgba format."""
    rgba = mcolors.to_rgba(color_name, opacity)
    return f'rgba({int(rgba[0]*255)}, {int(rgba[1]*255)}, {int(rgba[2]*255)}, {rgba[3]})'

## V. Main Work Flow

### Global Variables

In [78]:
# Global variables to store data processed in main

combined_df = None
ath_dict = {}
current_price_dict = {}
percent_change_ath_dict = {}

### Main Function

In [79]:
# Main Function

def main():
    global combined_df, ath_dict, current_price_dict 
    
    symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']
    
    # Fetch and save data
    df = fetch_and_save_all_time_data(symbols)
    combined_df = load_combined_data_from_csv()
    
    if combined_df is not None:
        ath_dict = {}
        current_price_dict = {}
        percent_change_ath_dict = {}

        for symbol in symbols:
            symbol_name = symbol[:-4]
            df_symbol = combined_df[combined_df['Symbol'] == symbol_name]

            # Calculate ATH
            ath = calculate_ath(symbol)
            ath_dict[symbol_name] = ath

            # Fetch current price
            current_price = round(fetch_current_price(symbol), 2)
            current_price_dict[symbol_name] = current_price

            # Calculate % change from ATH
            if current_price is not None:
                percent_change_ath_dict[symbol_name] = round(((current_price - ath) / ath) * 100, 2)
            else:
                percent_change_ath_dict[symbol_name] = None

        # Debug: Print out the contents of the dictionaries
        print("\nATH Dictionary:")
        for k, v in ath_dict.items():
            print(f"{k}: {v}")

        print("\nCurrent Price Dictionary:")
        for k, v in current_price_dict.items():
            print(f"{k}: {v}")

        print("\nPercent Change from ATH Dictionary:")
        for k, v in percent_change_ath_dict.items():
            print(f"{k}: {v}")

### Run Main

In [80]:
# Run Main

if __name__ == "__main__":
     main()

Data for BTCUSDT saved to data/BTCUSDT.csv
Data for ETHUSDT saved to data/ETHUSDT.csv
Data for BNBUSDT saved to data/BNBUSDT.csv
All data combined and saved to combined_data.csv

ATH Dictionary:
BTC: 110797.38
ETH: 4093.92
BNB: 761.0

Current Price Dictionary:
BTC: 111162.23
ETH: 2633.28
BNB: 680.5

Percent Change from ATH Dictionary:
BTC: 0.33
ETH: -35.68
BNB: -10.58


## VI. Figures

#### Resources for Styling

https://panel.holoviz.org/reference/panes/Plotly.html

https://panel.holoviz.org/how_to/styling/plotly.html



### Current Price vs. All-Time-High for All Symbols

In [81]:
# Current Price vs. All-Time-High for All Symbols

def plot_current_vs_ath(symbols, df, ath_dict, current_price_dict, colors_a, colors_b):
    """Plot the current price vs All-Time High."""
    fig = go.Figure()

    for symbol in symbols:
        symbol_name = symbol[:-4]
        df_symbol = df[df['Symbol'] == symbol_name]
        
        fig.add_trace(go.Bar(
            name=f'{symbol_name} All-Time-High: $ {ath_dict[symbol_name]:.2f}', # show Price in Legend
            y=[symbol_name],
            x=[ath_dict[symbol_name]],
            marker_color=convert_color(colors_b[symbol_name], 0.6),
            orientation='h',
            hovertemplate=f'{symbol_name}<br>All-Time-High: $ %{{x:.2f}}<extra></extra>'
        ))

        fig.add_trace(go.Bar(
            name=f'{symbol_name} Current Price: $ {current_price_dict[symbol_name]:.2f}',
            y=[symbol_name],
            x=[current_price_dict[symbol_name]],
            marker_color=convert_color(colors_a[symbol_name], 0.8),
            orientation='h',
            hovertemplate=f'{symbol_name}<br>Latest Price: $ %{{x:.2f}}<extra></extra>'
        ))

    fig.update_layout(
        title_text="Cryptocurrency Market Overview: Current Price vs. All-Time High",
        xaxis_title="Price (USD)",
        barmode='overlay',
        xaxis_type='log',
        template='plotly_white'
    )

    return fig

#### Test Figure Creation & Display

In [99]:
# Test Figure Creation 

fig_ath = plot_current_vs_ath(symbols, combined_df, ath_dict, current_price_dict, colors_a, colors_b)

In [100]:
# Test Display of Figure

plotly_fig1 = pn.pane.Plotly(fig_ath)

plotly_fig1.show()

Launching server at http://localhost:38829


QSocketNotifier: Can only be used with threads started with QThread


Wird in einer aktuellen Browsersitzung geöffnet.


### Stacked Price Curves for All Symbols

In [101]:
# Stacked Price Curves for All Symbols

def plot_price_comparison(symbols, df, colors_a, colors_b):
    """Plot the price curves over time for the given symbols."""
    fig = go.Figure()

    for symbol in symbols:
        symbol_name = symbol[:-4]
        df_symbol = df[df['Symbol'] == symbol_name]

        fig.add_trace(go.Scatter(
            x=df_symbol['Date'],
            y=df_symbol['Close'],
            mode='lines',
            name=f'{symbol_name} Close',
            line=dict(color=convert_color(colors_a[symbol_name], 0.8)),
            fill='tozeroy',
            fillcolor=convert_color(colors_b[symbol_name], 0.6),
            hovertemplate=f'{symbol_name}<br>Date: %{{x}}<br>Close: $ %{{y:.2f}}<extra></extra>'
        ))

    fig.update_layout(
        title_text="Cryptocurrency Prices Over Time",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        yaxis_type="log",
        xaxis_rangeslider_visible=True,
        template='plotly_white',
        height=700,
    )

    return fig

#### Test Figure Creation & Display

In [102]:
# Test Figure Creation

fig_stacked = plot_price_comparison(symbols, combined_df, colors_a, colors_b)

In [103]:
# Test Display of Figure

plotly_fig2 = pn.pane.Plotly(fig_stacked)

plotly_fig2.show()

Launching server at http://localhost:39621


QSocketNotifier: Can only be used with threads started with QThread


### Price Over Time

In [104]:
# Function to plot a specific symbol from pre-loaded data
def plot_price(symbol, df, colors_a, colors_b):
    """Plot the price curve over time for one given symbol."""
    
    # Check if 'Symbol' column exists
    if 'Symbol' not in df.columns:
        print(f"Column 'Symbol' not found in DataFrame for {symbol}.")
        return None
    
    # Filter data for the selected symbol
    df_symbol = df[df['Symbol'] == symbol]

    # Ensure symbol name matches dictionary keys (e.g., remove USDT if necessary)
    symbol_name = symbol.replace('USDT', '').strip()  # Ensure the symbol is properly formatted
    
    # Create the plot
    fig = go.Figure()

    # High-Low Band Plot
    fig.add_trace(go.Scatter(
        x=df_symbol['Date'],
        y=df_symbol['High'],
        mode='lines',
        name=f'{symbol_name} High',
        line=dict(color=colors_b[symbol_name], dash='dot'),
        hovertemplate=f'<br> High Price: $ %{{y:.2f}}<extra></extra>',
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x=df_symbol['Date'],
        y=df_symbol['Low'],
        mode='lines',
        name=f'{symbol_name} Low',
        line=dict(color=colors_b[symbol_name], dash='dot'),
        fill='tonexty',
        fillcolor=convert_color(colors_b[symbol_name], 0.4),
        hovertemplate=f'<br> Low Price: $ %{{y:.2f}}<extra></extra>',
        showlegend=False
    ))

    # Close Price Plot
    fig.add_trace(go.Scatter(
        x=df_symbol['Date'],
        y=df_symbol['Close'],
        mode='lines',
        name=f'{symbol_name} Close',
        line=dict(color=colors_a[symbol_name]),
        hovertemplate=f'<br> Close Price: $ %{{y:.2f}}<extra></extra>'
    ))

    # Open Price Plot (hidden)
    fig.add_trace(go.Scatter(
        x=df_symbol['Date'],
        y=df_symbol['Open'],
        mode='lines',
        name=f'{symbol_name} Open',
        line=dict(color=colors_b[symbol_name], dash='dash'),
        hovertemplate=f'<br> Open Price: $ %{{y:.2f}}<extra></extra>',
        showlegend=True,
        visible='legendonly'  # This hides the line but keeps it in the legend
    ))

    # 50-day SMA Plot
    fig.add_trace(go.Scatter(
        x=df_symbol['Date'],
        y=df_symbol['SMA_50'],
        mode='lines',
        name=f'{symbol_name} 50-day SMA',
        line=dict(color=convert_color(colors_a[symbol_name], 0.4)),
        hovertemplate=f'<br> 50-day SMA: $ %{{y:.2f}}<extra></extra>'
    ))

    # 200-day SMA Plot
    fig.add_trace(go.Scatter(
        x=df_symbol['Date'],
        y=df_symbol['SMA_200'],
        mode='lines',
        name=f'{symbol_name} 200-day SMA',
        line=dict(color=convert_color(colors_a[symbol_name], 0.6), dash='dash'),
        hovertemplate=f'<br> 200-day SMA: $ %{{y:.2f}}<extra></extra>'
    ))

    # 50-day EMA Plot
    fig.add_trace(go.Scatter(
        x=df_symbol['Date'],
        y=df_symbol['EMA_50'],
        mode='lines',
        name=f'{symbol_name} 50-day EMA',
        line=dict(color=convert_color(colors_b[symbol_name], 0.6)),
        hovertemplate=f'<br> 50-day EMA: $ %{{y:.2f}}<extra></extra>'
    ))

    # 200-day EMA Plot
    fig.add_trace(go.Scatter(
        x=df_symbol['Date'],
        y=df_symbol['EMA_200'],
        mode='lines',
        name=f'{symbol_name} 200-day EMA',
        line=dict(color=convert_color(colors_b[symbol_name], 0.8), dash='dash'),
        hovertemplate=f'<br> 200-day EMA: $ %{{y:.2f}}<extra></extra>'
    ))

    fig.update_layout(
        title=f'Cryptocurrency Price Over Time for {symbol_name}',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        yaxis_type='log',
        height=800,
        template='plotly_white',
        xaxis=dict(showgrid=True, showline=True, showticklabels=True, gridcolor='rgb(204, 204, 204)', linecolor='rgb(204, 204, 204)', linewidth=2, ticks='outside'),
        yaxis=dict(showgrid=True, showline=True, showticklabels=True, gridcolor='rgb(204, 204, 204)', linecolor='rgb(204, 204, 204)', linewidth=2, ticks='outside'),
        hovermode="x unified",
        xaxis_rangeslider_visible=True,
    )

    return fig

Wird in einer aktuellen Browsersitzung geöffnet.


#### Test Figure Creation & Display

In [105]:
# Test Display of Figure
symbol = 'BTC'
df_BTC = dataframes.get('BTC')  # Example of accessing the DataFrame for BTC
df_symbol = df_BTC
df_filtered = filter_outliers_percentiles(df_symbol, 'High') # Filter for Outliers if Necessary
price_BTC = plot_price(symbol, df_filtered, colors_a, colors_b) # Replace df_BTC with df_filtered and vice versa
plotly_fig3 = pn.pane.Plotly(price_BTC)
plotly_fig3.show()

Launching server at http://localhost:39547


QSocketNotifier: Can only be used with threads started with QThread


##### Price Figure ETH

In [106]:
symbol = 'ETH'
#dataframes = load_symbol_data(symbols, directory)
df_ETH = dataframes.get('ETH')  # Example of accessing the DataFrame for BTC
df_symbol = df_ETH
price_ETH = plot_price(symbol, df_ETH, colors_a, colors_b)

Wird in einer aktuellen Browsersitzung geöffnet.


##### Price Figure BNB

In [107]:
symbol = 'BNB'
#dataframes = load_symbol_data(symbols, directory)
df_BNB = dataframes.get('BNB')  # Example of accessing the DataFrame for BTC
df_symbol = df_BNB
price_BNB = plot_price(symbol, df_BNB, colors_a, colors_b)

#### Debug

In [108]:
# Debug

# Symbol List
#symbols = ['BTCUSDT', 'ETHUSDT']  # Add symbols for the cryptocurrencies you want to load

# Load Symbol Data
#dataframes = load_symbol_data(symbols)

# Check Columns (SMA, EMA present?)
#print(combined_df.columns)

# Check Symbol Column
# print(combined_df['Symbol'].unique())

#Pick Symbol and Filter DataFrame
#symbol_name = 'BTC'
#df_symbol_filtered = combined_df[combined_df['Symbol'] == symbol_name]

# Check Filtered Dataframe
#print(df_symbol_filtered.head())

### Volume Over Time

In [109]:
def plot_volume(symbol, df, colors_a, colors_b):
    """Plot the price volume curve over time for one given symbol."""
    
    # Check if 'Symbol' column exists
    if 'Symbol' not in df.columns:
        print(f"Column 'Symbol' not found in DataFrame for {symbol}.")
        return None
    
    # Filter data for the selected symbol
    df_symbol = df[df['Symbol'] == symbol]

    # Ensure symbol name matches dictionary keys
    symbol_name = symbol.replace('USDT', '').strip()
    
    # Create the plot
    fig = go.Figure()

    # Add Volume Plot
    fig.add_trace(go.Scatter(
        x=df_symbol['Date'],
        y=df_symbol['Volume'],
        mode='lines',
        name=f'{symbol_name} Volume',
        line=dict(color=convert_color(colors_b[symbol_name], 0.6)),  # Solid line for Volume
        fill='tozeroy',  # Fill area under the curve to the x-axis
        fillcolor=convert_color(colors_b[symbol_name], 0.6), 
        hovertemplate=f'{symbol_name}<br>Volume: $ %{{y:.2f}}<extra></extra>'
    ))

    # Configure Layout
    fig.update_layout(
        title=f'Volume for {symbol_name} Over Time',
        xaxis_title='Date',
        yaxis_title='Volume',
        yaxis_type='log',
        height=300,
        template='plotly_white',
        xaxis=dict(showgrid=True, showline=True, showticklabels=True, gridcolor='rgb(204, 204, 204)', linecolor='rgb(204, 204, 204)', linewidth=2, ticks='outside'),
        yaxis=dict(showgrid=True, showline=True, showticklabels=True, gridcolor='rgb(204, 204, 204)', linecolor='rgb(204, 204, 204)', linewidth=2, ticks='outside'),
        hovermode="x unified",
    )
    
    return fig

#### Test Figure Creation & Display

##### Volume Figure BTC

In [110]:
# Test Display of Figure
symbol = 'BTC'
#dataframes = load_symbol_data(symbols, directory)
df_BTC = dataframes.get('BTC')  # Example of accessing the DataFrame for BTC
df_symbol = df_BTC
volume_BTC = plot_volume(symbol, df_BTC, colors_a, colors_b)
plotly_fig3a = pn.pane.Plotly(volume_BTC)
plotly_fig3a.show()

Launching server at http://localhost:36737


QSocketNotifier: Can only be used with threads started with QThread


##### Volume Figure (ETH)

In [111]:
symbol = 'ETH'
#dataframes = load_symbol_data(symbols, directory)
df_ETH = dataframes.get('ETH')  # Example of accessing the DataFrame for BTC
df_symbol = df_ETH
volume_ETH = plot_volume(symbol, df_ETH, colors_a, colors_b)

Wird in einer aktuellen Browsersitzung geöffnet.


##### Volume Figure (BNB)

In [112]:
symbol = 'BNB'
#dataframes = load_symbol_data(symbols, directory)
df_BNB = dataframes.get('BNB')  # Example of accessing the DataFrame for BTC
df_symbol = df_BNB
volume_BNB = plot_volume(symbol, df_BNB, colors_a, colors_b)

## VI. Dashboard (Test)

### Resources

**General**

https://panel.holoviz.org/

**Plotly**

https://panel.holoviz.org/reference/panes/Plotly.html

**Layout**

https://panel.holoviz.org/how_to/notebook/layout_builder.html

### Components

#### Get Dataframes for  (get_crypto_data)

In [113]:
def get_crypto_data(symbol):
    """Fetch the dataframe for the selected cryptocurrency symbol."""
    df_symbol = dataframes.get(symbol)
    return df_symbol

#### Symbol Selection (select_view)

In [114]:
# Initialize the select_view widget

select_view = pn.widgets.Select(name='Select Cryptocurrency:', options=[symbol[:-4] for symbol in symbols], value='BTC')

#### Add Symbol

In [115]:
# Add symbols to list

#### Figures (create_price_volume_figs)

In [116]:
def create_price_volume_figs(symbol):
    """Generate the price and volume figures for the selected cryptocurrency."""
    df_symbol = get_crypto_data(symbol)

    # Generate price figure
    price_fig = plot_price(symbol, df_symbol, colors_a, colors_b)
    
    # Generate volume figure
    volume_fig = plot_volume(symbol, df_symbol, colors_a, colors_b)

    # Return both figures wrapped in panel objects
    return pn.pane.Plotly(price_fig, height=800, sizing_mode="stretch_width"), pn.pane.Plotly(volume_fig, height=300, sizing_mode="stretch_width")

##### Manual creation of Figures

In [117]:
# # Comparison
# ath_bars = pn.pane.Plotly(fig_ath, height=500, sizing_mode="stretch_width").servable()
# stacked_comparison = pn.pane.Plotly(fig_stacked, height=500, sizing_mode="stretch_width").servable()

# # BTC
# price_fig_BTC = pn.pane.Plotly(price_BTC, height=800, sizing_mode="stretch_width").servable()
# volume_fig_BTC = pn.pane.Plotly(volume_BTC, height=300, sizing_mode="stretch_width").servable()

# # ETH
# price_fig_ETH = pn.pane.Plotly(price_ETH, height=800, sizing_mode="stretch_width").servable()
# volume_fig_ETH = pn.pane.Plotly(volume_ETH, height=300, sizing_mode="stretch_width").servable()

# # BNB
# price_fig_BNB = pn.pane.Plotly(price_BNB, height=800, sizing_mode="stretch_width").servable()
# volume_fig_BNB = pn.pane.Plotly(volume_BNB, height=300, sizing_mode="stretch_width").servable()

### Dashboard Creation

In [124]:
@pn.depends(select_view.param.value)
def update_dashboard(symbol):
    """Update the dashboard based on the selected cryptocurrency symbol."""
    # df_symbol = combined_df[combined_df['Symbol'] == symbol]
    # df_symbol = get_crypto_data(symbol)
    ath = ath_dict.get(symbol, 0)
    current_price = current_price_dict.get(symbol, 0)

    # Check if values are floats; otherwise, convert or set to 0
    try:
        ath = float(ath)
    except ValueError:
        ath = 0

    try:
        current_price = float(current_price)
    except ValueError:
        current_price = 0
    
    price_fig, volume_fig = create_price_volume_figs(symbol)
    
    return pn.Column(
        price_fig,
        volume_fig
    )

dashboard = pn.Column(
    pn.pane.Markdown("# Cryptocurrency Dashboard"),
    select_view,
    update_dashboard
)
dashboard.servable();

In [125]:
dashboard.show()

Launching server at http://localhost:43681


QSocketNotifier: Can only be used with threads started with QThread


Wird in einer aktuellen Browsersitzung geöffnet.


### App (Test)

In [ ]:
# pn.template.FastListTemplate(
#     title="Cryptocurrency Dashboard",
#     sidebar=[select_view],
#     main=[dashboard], #change to fig_price & fig_volume
#     main_layout=None,
#     #accent=ACCENT, #create variable ACCENT = <THEME>
# ).servable();